# Spectral Clustering of Countries Economic Development

## Introduction
This notebook aims to investigate if spectral clustering can be applied for the task of grouping the financial growth of 113 countries over the past 58 years. The whole process is documented step by step.

### Imports

In [ ]:
import numpy as np

import csv

import warnings

from tslearn.clustering import silhouette_score

import Constants

from Utils import DataUtils, PreProcessing, VisualUtils, TimeSeriesUtils, PostProcessing

from Clustering import SpectralClustering, KernelKMeans

### Data
For each country, three different time series of data are considered; the annual evolution of its GDP per capita, the annual evolution of its population and the annual evolution of the bilateral exchange rate of its currency against the US Dollar. While there is (partially) available data for the GDP and population evolution all the way from 1900, data about currency exchange rates is only available since 1960. For all three time series, the latest data concerns 2017. In order to stabilize the dataset, all GDP and population data before 1960 is discarded.

Because the GDP time series present very high variance, the logarithm of the series is taken instead to smoothen the results.

The data for each time series is loaded from a `.csv` file to create a $113 \times T$ matrix, where $T$ is the amount of years each time series is available for.

In [ ]:
names, gdp, pop, currency, map = DataUtils.load_clustering_data()
locations = DataUtils.load_locations()
groups = DataUtils.load_groups()
gdp_data = np.log(gdp[:, -Constants.T:])
pop_data = pop[:, -Constants.T:]

## Data Preparation
Before the clustering algorithms are applied, the time series data is concatenated for all countries and then standardized.

One complication that arises for currency exchange rate data when it comes to countries of former Yugoslavia and USSR. For these countries (Serbia, Croatia, Bosnia and Herzegovina and Russia respectively) no data is available until 1990. To tackle this problem, a nearest neighbor imputer is used to fill the empty values.

In [ ]:
df, scaled_df = PreProcessing.preprocess(
    names, gdp_data, pop_data, currency
)

In [ ]:
df_gdp, scaled_df_gdp, scaled_data_gdp = PreProcessing.preprocess_onlyGDP(names, gdp_data)

## Spectral Clustering

We examine three different ways of building the similarity graph; a euclidean $k$-nearest neighbor graph, an $\epsilon$-neighborhood graph and a DTW $k$-nearest neighbor graph.

In order to tune the number of clusters for the algorithm, the 'eigenvalue' heuristic is used. Specifically, we examine the magnitude of the eigenvalues of the Laplacian matrix of the graph $L$, and take K clusters such that the smallest eigenvalues $\lambda_1$, ..., $\lambda_K$ are significantly smaller than the rest. Afterwards, we examine the information captured by the corresponding eigenvectors. We reduce the number of clusters by one for each eigenvector that does not capture any significant information.

### $k$-nearest neighbor graph:

We use $k=\lfloor \sqrt{n} \rfloor$ neighbors, where $n$ is the number of countries, i.e. 113. 

In [ ]:
W_knn = SpectralClustering.knn_graph(scaled_df)

In [ ]:
eigvals_knn_norm, eigvecs_knn_norm = SpectralClustering.laplacian_eigen(
    W_knn, True, True
)

We conclude 3 clusters are sufficient. We create the H matrix:

In [ ]:
K_knn = 3
H_knn = SpectralClustering.smallest_eigenvecs(
    K_knn, eigvals_knn_norm, eigvecs_knn_norm, True
)

All of the eigenvectors are relevant, so now we can perform the clustering with 3 clusters

In [ ]:
labels_knn = SpectralClustering.kmeans(
    H_knn, 3
)

score_knn = silhouette_score(scaled_data_gdp, labels_knn)
cluster_centers_knn = TimeSeriesUtils.cluster_centroids(scaled_data_gdp, 3, labels_knn)
clusters = VisualUtils.show_clustering(
    names, 
    3, 
    scaled_data_gdp, 
    cluster_centers_knn, 
    labels_knn, 
    score_knn, 
    2, 
    2,
    "Spectral Clustering with Euclidean k-neighbors"
)

### $\epsilon$-neighborhood graph:

In order for the similarity graph to be sparse, we set $\epsilon$ to the 10th percentile of the distances.

In [ ]:
W_eps = SpectralClustering.epsilon_graph(
    scaled_df, 10, True
)

In [ ]:
eigvals_eps_norm, eigvecs_eps_norm = SpectralClustering.laplacian_eigen(
    W_eps, True, True
)

We conclude 21 clusters are sufficient. We create the H matrix:

In [ ]:
K_eps = 21
H_eps = SpectralClustering.smallest_eigenvecs(
    K_eps, eigvals_eps_norm, eigvecs_eps_norm, True
)

Two different eigenvectors give no information, so we can reduce the number of clusters to 19:

In [ ]:
labels_eps = SpectralClustering.kmeans(
    H_eps, K_eps-2
)
score_eps = silhouette_score(scaled_data_gdp, labels_eps)
cluster_centers_eps = TimeSeriesUtils.cluster_centroids(scaled_data_gdp, K_eps-2, labels_eps)
clusters = VisualUtils.show_clustering(
    names, 
    K_eps-2, 
    scaled_data_gdp, 
    cluster_centers_eps, 
    labels_eps, 
    score_eps, 
    4, 
    5,
    "Spectral Clustering with epsilon neighborhood"
)

### Spectral Clustering, Dynamic Time Warping and Kernel $k$-Means

Here, we construct a $k$-neighbors graph similar to the one before, but with a different distance measure. Instead of Euclidean Distance, Dynamic Time Warpint (DTW) is used to better capture the similaritites and differences of time series. Instead of the concatenated dataset, this section only considers the GDP time series, as it is the one of the highest significance and variance.

As suggested by previous work, this version of spectral clustering can provide an initial partition to be used for spectral clustering. For this reason, we have predetermined the number of clusters to 6, as this is the optimal number of clusters for kernel k-means according to the elbow plot heuristic. We can see, however, that the eigenvalue plot indicates 6 is a reasonable choice, as there is a clear gap between the 6th and 7th smallest eigenvalues, and all 6 corresonding eigenvectors are informative. 

In [14]:
W_dtw = SpectralClustering.dtw_knn_graph(n, scaled_df_gdp)

In [ ]:
eigvals_dtw_norm, eigvecs_dtw_norm = SpectralClustering.laplacian_eigen(
    n, W_dtw, True, True
)

In [ ]:
K_dtw = 6
H_dtw = SpectralClustering.smallest_eigenvecs(
    K_dtw, eigvals_dtw_norm, eigvecs_dtw_norm, True
)

In [ ]:
labels_dtw = SpectralClustering.kmeans(
    H_dtw, K_dtw
)

score_dtw = silhouette_score(scaled_data_gdp, labels_dtw)
cluster_centers_dtw = TimeSeriesUtils.cluster_centroids(scaled_data_gdp, K_dtw, labels_dtw)
clusters = VisualUtils.show_clustering(
    names, 
    K_dtw, 
    scaled_data_gdp, 
    cluster_centers_dtw, 
    labels_dtw, 
    score_dtw, 
    3, 
    2,
    "Spectral Clustering with DTW"
)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    kkmeans = KernelKMeans.NonRandomKernelKMeans(6, labels_dtw)
    y_kkmeans = kkmeans.fit_predict(scaled_df_gdp)

score_kkmeans = silhouette_score(scaled_data_gdp, y_kkmeans)
# cluster_centers_kkmeans = TimeSeriesUtils.cluster_centroids(scaled_data_gdp, K_dtw, y_kkmeans, T_currency)
clusters = VisualUtils.show_clustering(
    names, 
    K_dtw, 
    gdp_data, 
    None, 
    y_kkmeans, 
    score_kkmeans, 
    3, 
    2, 
    "Kernel 6-Means with Spectral Clustering Initialization"
)

This algorithm produces well-structured clusters and achieves a considerable silhouette score. We visualize the results on a map and we display how the clustering corresponds to commonly accepted groupings of countries

In [ ]:
VisualUtils.show_clusters_on_map(names, y_kkmeans, map, 'Kernel 6-Means with GAK and Spectral Clustering Initialization')

In [ ]:
PostProcessing.postprocess_clustering(scaled_data_gdp, locations, names, groups, y_kkmeans)

In [21]:
# with open(labels_spec_path, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Country', 'Cluster'])
#     for i, name in enumerate(names):
#         writer.writerow([name, y_kkmeans[i]])

In [22]:
# DataUtils.write_data(scaled_data_gdp, scaled_gdp_path)

## Conclusion
While spectral clustering on its own does not produce desirable results, it can enhance the performance of kernel $k$-means to produce the best partition of all the examined algorithms.